In [ ]:
import pandas as pd
import json
import urllib.request as urlreq
import os

In [ ]:
path_to_data = os.path.join(os.getcwd(),'University of Missouri','DRIVERS','data')
path_to_data

In [ ]:
spark.sql("use real_world_data_ed_omop_dec_2023")

In [ ]:
cd_meta = spark.sql('''
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'DRG' and
          concept_code in (
            '765','766','767','768',
            '774','775',
            '783','784','785','786', '787','788',
            '796','797','798',
            '805','806','807'
          )
    union all 
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'CPT4' and
          concept_code in (
            '59409','59514', '59612','59620'
          )
    union all 
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'ICD10PCS' and
          concept_code in (
            '10D00Z0','10D00Z1','10D00Z2','10D07Z3','10D07Z4', '10D07Z5', '10D07Z6','10D07Z7','10D07Z8',
            '10E0XZZ'
          )
    union all
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'LOINC' and
          concept_code in (
            
          )
''').toPandas()
cd_meta.to_csv(os.path.join(path_to_data,'cd_meta_omop.csv'),index=False)

In [ ]:
def cd_where_clause(meta_tbl,voc_id):
    '''
    generate where clause details based on omop concept_id
    '''
    cd_lst = meta_tbl.loc[meta_tbl['vocabulary_id']==voc_id,'concept_id'].tolist()
    cd_quote = []
    for code in cd_lst:
        cd_quote.append("'"+ str(code) +"'")
    cd_quote_str = ",".join(cd_quote)
    return cd_quote_str     

meta_tbl = pd.read_csv(os.path.join(path_to_data,'cd_meta_omop.csv'))
drg_where = cd_where_clause(meta_tbl,'DRG')
cpt4_where = cd_where_clause(meta_tbl,'CPT4')
icd10_where = cd_where_clause(meta_tbl,'ICD10PCS')